In [12]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('explore_spark').config('spark.master', 'local').getOrCreate()

In [2]:
ps.set_option('display.max_rows', 10)

In [11]:
prev = spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")  # Keep its default value.
ps.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

In [18]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
%timeit ps.range(300000).to_pandas()

117 ms ± 38.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
name_basics = ps.read_csv('../Data/name.basics.tsv', sep='\t', inferSchema=True)
# title_akas = ps.read_csv('../Data/title.akas.tsv', sep='\t', inferSchema=True)
title_basics = ps.read_csv('../Data/title.basics.tsv', sep='\t', inferSchema=True)
title_crew = ps.read_csv('../Data/title.crew.tsv', sep='\t', inferSchema=True)
# title_episode = ps.read_csv('../Data/title.episode.tsv', sep='\t', inferSchema=True)
title_principals = ps.read_csv('../Data/title.principals.tsv', sep='\t', inferSchema=True)
title_ratings = ps.read_csv('../Data/title.ratings.tsv', sep='\t', inferSchema=True)

In [10]:
type(name_basics)

pyspark.pandas.frame.DataFrame

In [4]:
name_basics.head(5)
# title_akas.head(5)
# title_basics.head(5)
# title_crew.head(5)
# title_episode.head(5)
# title_principals.head(5)
# title_ratings.head(5)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0072308,tt0031983,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0075213,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0056404,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0077975,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0050986,tt0050976,tt0069467"


In [23]:
# title_ratings.describe()

In [22]:
title_ratings.sort_values(by='numVotes', ascending=False).head()

,tconst,averageRating,numVotes
82643,tt0111161,9.3,2824743
252047,tt0468569,9.0,2806239
650347,tt1375666,8.8,2490927
99224,tt0137523,8.8,2260236
438449,tt0944947,9.2,2226899


In [25]:
title_basics.groupby('titleType').count().sort_values('tconst')['tconst']

titleType
tvPilot              1
tvShort          10084
videoGame        36627
tvSpecial        44591
tvMiniSeries     51367
tvMovie         143596
tvSeries        253097
video           283511
movie           664652
short           964958
Name: tconst, dtype: int64
Showing only the first 10

In [33]:
title_basics_filtered = title_basics.loc[title_basics['titleType'] == 'movie'].drop(columns=['endYear']).replace('\\N', np.nan)
title_basics_filtered

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,0,1905,100,NaN
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,Drama
625,tt0000630,movie,Hamlet,Amleto,0,1908,NaN,Drama
668,tt0000675,movie,Don Quijote,Don Quijote,0,1908,NaN,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"
828,tt0000838,movie,A Cultura do Cacau,A Cultura do Cacau,0,1909,NaN,NaN


In [40]:
title_basics_filtered['genres'].isnull()

8      False
144    False
498    False
570    False
587    False
610    False
625    False
668    False
672    False
828    False
Name: genres, dtype: bool
Showing only the first 10

In [36]:
title_basics_filtered.isnull().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
runtimeMinutes    0
genres            0
dtype: int64

In [37]:
title_basics_filtered.isna().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
runtimeMinutes    0
genres            0
dtype: int64

## Concatenate stuff

In [41]:
title_basics['titleType'].unique()

0        tvSeries
1    tvMiniSeries
2         tvMovie
3       tvEpisode
4           movie
5       tvSpecial
6           video
7       videoGame
8         tvShort
9           short
Name: titleType, dtype: object
Showing only the first 10

In [44]:
# title_basics = title_basics.loc[title_basics['titleType'] == 'movie']
# # title_basics = title_basics.loc[title_basics['genres'] != '\\N']
# title_basics = title_basics.to_pandas().loc[title_basics['tconst'].isin(title_ratings['tconst'].to_pandas())]

PandasNotImplementedError: The method `pd.Series.__iter__()` is not implemented. If you want to collect your data as an NumPy array, use 'to_numpy()' instead.